# CNN을 활용한 음식 사진 이름 예측 모델 구현

In [12]:
import os
import numpy as np
from unidecode import unidecode
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

data_dir = "./korean_food"

#폴더명 한글에서 영어 변환
for folder in os.listdir(data_dir):
    src = os.path.join(data_dir, folder)
    if os.path.isdir(src):
        eng_name = unidecode(folder).lower().replace(" ", "_")
        dst = os.path.join(data_dir, eng_name)
        if not os.path.exists(dst):
            os.rename(src, dst)
            print(f"{folder} -> {eng_name} 변경 완료")
        else:
            print(f"{dst} 건너뜀")

#비영상 데이터 제거
for root, dirs, files in os.walk(data_dir):
    for f in files:
        if not f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
            os.remove(os.path.join(root, f))
            print(f"❌ 제거: {os.path.join(root, f)}")

./korean_food\aehobagboggeum 건너뜀
./korean_food\albab 건너뜀
./korean_food\baecugimci 건너뜀
./korean_food\baeggimci 건너뜀
./korean_food\bibimbab 건너뜀
./korean_food\bibimnaengmyeon 건너뜀
./korean_food\bossam 건너뜀
./korean_food\bucugimci 건너뜀
./korean_food\bugeosgug 건너뜀
./korean_food\bulgogi 건너뜀
./korean_food\conggaggimci 건너뜀
./korean_food\cueotang 건너뜀
./korean_food\dalgboggeumtang 건너뜀
./korean_food\dalggalbi 건너뜀
./korean_food\dalggyejang 건너뜀
./korean_food\ddangkongjorim 건너뜀
./korean_food\ddeogboggi 건너뜀
./korean_food\ddeoggalbi 건너뜀
./korean_food\ddeogggoci 건너뜀
./korean_food\ddeoggug_mandugug 건너뜀
./korean_food\deodeoggui 건너뜀
./korean_food\doenjangjjigae 건너뜀
./korean_food\donggeurangddaeng 건너뜀
./korean_food\dongtaejjigae 건너뜀
./korean_food\dubugimci 건너뜀
./korean_food\dubujorim 건너뜀
./korean_food\eomugboggeum 건너뜀
./korean_food\gajiboggeum 건너뜀
./korean_food\galbigui 건너뜀
./korean_food\galbijjim 건너뜀
./korean_food\galbitang 건너뜀
./korean_food\galcigui 건너뜀
./korean_food\galcijorim 건너뜀
./korean_food\gamjacaebogg

In [15]:
#2. 데이터셋 불러오기

img_size = (224, 224)
batch_size = 32
seed = 123

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)


Found 150507 files belonging to 150 classes.
Using 120406 files for training.
Found 150507 files belonging to 150 classes.
Using 30101 files for validation.


In [18]:
#모델 정의 및 학습

normalization_layer = layers.Rescaling(1./255)
base_model = keras.applications.EfficientNetB0(
    input_shape=img_size + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # feature extractor만 사용

model = keras.Sequential([
    normalization_layer,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(len(class_names), activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Epoch 1/10
 818/3763 ━━━━━━━━━━━━━━━━━━━━ 21:50 445ms/step - accuracy: 0.0067 - loss: 5.1278

InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_48844]

In [ ]:
#모델 평가 및 예측
# 자동 역매핑: 영어 폴더명 → 한글 폴더명
eng2kor = {unidecode(k).lower().replace(" ", "_"): k for k in os.listdir(data_dir)}

def predict_image(model, img_path):
    from tensorflow.keras.utils import load_img, img_to_array
    img = load_img(img_path, target_size=img_size)
    x = img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)
    pred = model.predict(x)
    pred_class = train_ds.class_names[np.argmax(pred)]
    return eng2kor.get(pred_class, pred_class)

In [ ]:
img_path = "Img_051_0971.jpg"
print("예측 음식:", predict_image(model, img_path))